<a href="https://www.kaggle.com/code/argada/bike-sharing-eda-visualisation-and-forecast?scriptVersionId=173931471" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

This Jupyter Notebook is designed to provide a comprehensive analysis of a Bike Sharing Dataset, using various data science techniques to understand the data's characteristics such as visualizations, clustering, and predictive modeling. The notebook is divided into several sections, each focusing on a specific aspect of the dataset and analysis.

### Section 1: Exploring the Dataset
**Objective**: Understand the dataset's format, content, and initial quality. <br>
**Tasks**: Perform an initial data quality test to check for abnormal values and identify any missing hours in the dataset. <br>
**Outputs**: Heatmaps illustrating 'cnt' values to identify missing data. 
<img src="https://i.postimg.cc/c15Mrbm3/results-32-0.png" width="900"/>


### Section 2: Basic Statistics
**Objective**: Generate basic statistical measures for key attributes. <br>
**Tasks**: Calculate range, mean, and standard deviation for attributes such as temperature, humidity, windspeed, and bike rental counts (casual, registered, and total). <br>
**Outputs**: Descriptive statistics table.

### Section 3: Visualizations
**Objective**: Visualize data distributions and correlations between attributes. <br>
**Tasks**: Create histograms for selected columns and correlation matrices. Explore the distribution of bike rentals across different seasons and weather conditions, and analyze the ratio of casual to registered rides over time. <br>
**Outputs**: Various figures illustrating data distributions and correlations.
<img src="https://i.postimg.cc/G2mGpFTz/results-43-0.png" width="900"/>


### Section 4: Clustering
**Objective**: Identify patterns in the data through clustering. <br>
**Tasks**: Use K-means clustering on the dataset to find patterns based on features like season, weather conditions, and time.<br>
**Outputs**: Clusters visualized on quantitative components and original features.
<img src="https://i.postimg.cc/J0yXNRqk/results-60-4.png" width="900"/>
<img src="https://i.postimg.cc/hjZxQX9n/results-60-5.png" width="900"/>

### Section 5: Prediction Model
**Objective**: Develop and evaluate a predictive model for bike rental counts.<br>
**Tasks**: Implement a Deep Neural Network (DNN) to predict casual and registered bike rental counts. Evaluate the model's performance using mean absolute error (MAE).<br>
**Outputs**: Comparison of true values versus model predictions. Cross-validation and baseline comparison metrics.<br>

<img src="https://i.postimg.cc/zGTn2X1w/results-78-0.png" width="400"/><img src="https://i.postimg.cc/TPGnzMRz/results-80-0.png" width="400"/><img src="https://i.postimg.cc/wTy5JQfG/results-79-0.png" width="400"/> 

In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import math
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import MonthLocator, DateFormatter
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import LogNorm, Normalize
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset

import scipy.stats as stats
from matplotlib.lines import Line2D 

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [ ]:
path_to_bikes = "/kaggle/input/rental-bike-sharing/hour.csv"
try:
    bike_df = pd.read_csv(path_to_bikes)
    print("First few rows of the Hour dataset")
    print(bike_df.head())
    print("\nData info:")
    print(bike_df.info())
except Exception as e:
    print("Error reading CSV file:", e)
    

# Quick data quality test
+ Identify missing data 
+ Check for ubnormal behavioural 

In [ ]:
try:
    assert  all(bike_df['instant'] == [i for i in range(1, len(bike_df)+1)])
    if all(bike_df['instant'] == [i for i in range(1, len(bike_df)+1)]):
        print("The 'instant' check")
    else:
        print("Make another check, something is wrong")
except AttributeError:
    print("AttributeError")

In [ ]:
bike_df['dteday'] = pd.to_datetime(bike_df['dteday']).dt.date

start_date = pd.to_datetime('2011-01-01').date()
end_date = pd.to_datetime('2012-12-31').date()
date_range = pd.date_range(start=start_date, end=end_date, freq='D').date
missing = [date for date in date_range if date not in bike_df['dteday'].values]

if not missing:
    print("The 'dteday' check")
else:
    print("Missing dates:", missing_dates)

In [ ]:
try:
    assert (bike_df['season'].unique() == [i for i in range(1,5)]).all()
    if (bike_df['season'].unique() == [i for i in range(1,5)]).all():
        print("The number of 'season's check")
    else:
        print("Make another check, something is wrong")
except AttributeError:
    print("AttributeError")

In [ ]:
bike_df['dteday'] = pd.to_datetime(bike_df['dteday'])
changing_season_dates = bike_df[bike_df['season'] != bike_df['season'].shift(1)]
print("Dates where 'season' is changing:")
print(changing_season_dates[['dteday', 'season']])

In [ ]:
try:
    assert (bike_df['yr'].unique() == (0, 1)).all()
    yr_1 = (bike_df['yr'].unique() == (0, 1)).all()
    yr_1 *= (bike_df[bike_df['dteday'].dt.year == 2011]['yr'].unique() == 0).all()
    yr_1 *= (bike_df[bike_df['dteday'].dt.year == 2012]['yr'].unique() == 1).all()
    if yr_1:
        print("The 'yr' check")
    else:
        print("Make another check, something is wrong")
except AttributeError:
    print("AttributeError")

In [ ]:
if (bike_df['mnth'] == bike_df['dteday'].dt.month).all():
    print("The 'mnth' check")
else:
    print("Make another check, something is wrong")

In [ ]:
#Must contain at least 4 anomalies for "daylight saving time" in US
print(bike_df['hr'].value_counts().unique())

In [ ]:
bike_df['dteday'] = pd.to_datetime(bike_df['dteday'])
hours_per_day = bike_df['dteday'].value_counts()
hours_per_day.value_counts()

No days with > 24, those end of "daylight saving time" can be added to anomalies 

In [ ]:
data = bike_df[['hr', 'dteday']]
presence_matrix = data.pivot(index='dteday', columns='hr', values='hr').notna()
presence_matrix = presence_matrix.iloc[:, ::-1]
presence_matrix = presence_matrix.T
cmap = sns.color_palette(["red", '#23B6EA'])

plt.figure(figsize=(15, 6))
sns.heatmap(presence_matrix, cmap=cmap, cbar=False)

N = 10  # Change N to control the spacing of x-axis labels
def format_fn(value, tick_number):
    if int(value) % N == 0:
        return presence_matrix.columns[int(value)].strftime('%Y-%m-%d')
    else:
        return ""
ax = plt.gca()
ax.xaxis.set_major_formatter(FuncFormatter(format_fn))
plt.title("Hours per day (Blue: Recorded, Red: Absent)")
plt.xlabel("Day (dteday)")
plt.ylabel("Hour (hr)")
plt.xticks(rotation=45)
plt.show()

In [ ]:
try:
    assert (bike_df['holiday'].unique() == (0,1)).all()
    if (bike_df['holiday'].unique() == (0,1)).all():
        print("The 'holiday' check")
    else:
        print("Make another check, something is wrong")
except AttributeError:
    print("AttributeError")

In [ ]:
bike_df['dteday'] = pd.to_datetime(bike_df['dteday'])
calculated_weekday = bike_df['dteday'].dt.dayofweek
weekday = (bike_df['weekday'] == bike_df['dteday'].dt.dayofweek).all()
if weekday:
    print("The 'weekday' check")
else:
    print("Make another check, something is wrong")

In [ ]:
(bike_df['dteday'].dt.dayofweek - bike_df['weekday']).unique()

In [ ]:
bike_df['dteday'] = pd.to_datetime(bike_df['dteday'])
calculated_weekday = bike_df['dteday'].dt.dayofweek
weekday = bike_df['weekday'] == bike_df['dteday'].dt.dayofweek + 1
weekday += bike_df['weekday'] == bike_df['dteday'].dt.dayofweek - 6
if weekday.all():
    print("The 'weekday' check")
else:
    print("Make another check, something is wrong")

In [ ]:
try:
    assert (bike_df['workingday'].unique() == (0,1)).all()
    if (bike_df['workingday'].unique() == (0,1)).all():
        print("The 'holiday' check")
    else:
        print("Make another check, something is wrong")
except AttributeError:
    print("AttributeError") 

In [ ]:
is_workingday = bike_df['workingday'] == 1
is_holidays = bike_df['holiday'] == 1
is_day56 = (bike_df['dteday'].dt.dayofweek == 5) | (bike_df['dteday'].dt.dayofweek == 6)
(is_holidays + is_day56 != is_workingday).all()

In [ ]:
bike_df['weathersit'].unique()

In [ ]:
np.min(bike_df['temp'].unique()), np.max(bike_df['temp'].unique())  #*41

In [ ]:
np.min(bike_df['atemp'].unique()), np.max(bike_df['atemp'].unique()) #*50

In [ ]:
np.min(bike_df['hum'].unique()), np.max(bike_df['hum'].unique()) #*100

In [ ]:
np.min(bike_df['windspeed'].unique()), np.max(bike_df['windspeed'].unique()) #*67

In [ ]:
try: 
    bike_df['casual'].astype(int)
    print("Integer")
except ValueError:
    print("Something wrong")

In [ ]:
np.min(bike_df['casual'].unique()), np.max(bike_df['casual'].unique())

In [ ]:
try: 
    bike_df['registered'].astype(int)
    print("Integer")
except ValueError:
    print("Something wrong")

In [ ]:
np.min(bike_df['registered'].unique()), np.max(bike_df['registered'].unique())

In [ ]:
try: 
    bike_df['cnt'].astype(int)
    print("Integer")
except ValueError:
    print("Something wrong")

In [ ]:
np.min(bike_df['cnt'].unique()), np.max(bike_df['cnt'].unique())

In [ ]:
((bike_df['registered'] + bike_df['casual']) == bike_df['cnt']).all()

In [ ]:
data = bike_df[['hr', 'dteday', 'cnt']]
presence_matrix = data.pivot(index='dteday', columns='hr', values='cnt') # .fillna(0)
presence_matrix = presence_matrix.iloc[:, ::-1]
presence_matrix = presence_matrix.T

cmap = sns.color_palette("light:#23B6EA", 100, as_cmap=True)
cmap.set_bad("red")

fig, ax = plt.subplots(figsize=(15, 6))
sns.heatmap(presence_matrix, cmap=cmap, cbar=True, norm=LogNorm())

N = 10  
def format_fn(value, tick_number):
    if int(value) % N == 0:
        return presence_matrix.columns[int(value)].strftime('%Y-%m-%d')
    else:
        return ""
ax = plt.gca()
ax.xaxis.set_major_formatter(FuncFormatter(format_fn))
plt.title("'cnt' per day (red: hour is absent)")
plt.xlabel("Day (dteday)")
plt.ylabel("Hour (hr)")
plt.xticks(rotation=45)
plt.savefig("Hour_Day_cnt.png", bbox_inches='tight')
plt.show() 

In [ ]:
data = bike_df[['hr', 'dteday', 'cnt']]
presence_matrix = data.pivot(index='dteday', columns='hr', values='cnt') #.notna()
presence_matrix = presence_matrix.iloc[:, ::-1]
presence_matrix = presence_matrix.T

cmap = sns.color_palette("light:#23B6EA", 100, as_cmap=True)
cmap.set_bad("red")

fig, ax = plt.subplots(figsize=(15, 6))
sns.heatmap(presence_matrix, cmap=cmap, cbar=True, norm=LogNorm())

N = 10 
def format_fn(value, tick_number):
    if int(value) % N == 0:
        return presence_matrix.columns[int(value)].strftime('%Y-%m-%d')
    else:
        return ""
ax = plt.gca()
ax.xaxis.set_major_formatter(FuncFormatter(format_fn))

plt.title("'cnt' per day (red: hour is absent)")
plt.xlabel("Day (dteday)")
plt.ylabel("Hour (hr)")
plt.xticks(rotation=45)

axins = zoomed_inset_axes(ax, 2, loc='upper right', borderpad=2)
mark_inset(ax, axins, loc1=2, loc2=3, ec="0.5")

axins.set_xlim([0, 80])  
axins.set_ylim([10, 23]) 

axins.matshow(presence_matrix, cmap=cmap, origin = 'upper',  extent = [0, 750, 10, 32], aspect="auto", norm=LogNorm())
axins.set_xticks([])
axins.set_yticks([])
#plt.invert_yaxis()
plt.show()

## Basic statistics of the data

In [ ]:
statistics_dict = {}
for column_name in bike_df.columns[-7:]:
    column = bike_df[column_name]
    statistics = {
        'Range': (column.min(), column.max()),
        'Mean': column.mean(),
        'STD': column.std(),
        '25%': column.quantile(0.25),
        '50%': column.quantile(0.5),
        '75%': column.quantile(0.75),
    }
    statistics_dict[column_name] = statistics
statistics_df = pd.DataFrame(statistics_dict)
print("Basic Statistics:")
print(statistics_df)

In [ ]:
def get_basic(column):
    selected_column = bike_df[column]
    statistics = selected_column.describe()
    print("Basic Statistics for column '{}'".format(column))
    print(statistics)
    print("")
    
for i in bike_df.columns:
    get_basic(i)

# Univariate visualization

In [ ]:
import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

bike_df = pd.read_csv(path_to_bikes)
mapping = {
    'holiday': {0: "no", 1: "yes"},
    'workingday': {0: "no", 1: "yes"},
    'weathersit' : {1 : "Clear", 2 : "Mist", 3 : "Light precip.", 4 : "High precip."}, 
    'season': {1: "Winter", 2: "Spring", 3: "Summer", 4: "Autumn"},
    'weekday': {0: "Sunday", 1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday"}
}
for i in mapping.keys():
    bike_df[i] = bike_df[i].map(mapping[i])
    
columns_to_plot = [i for i in bike_df.columns[8:]]
columns_to_plot.insert(0, bike_df.columns[6])

fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(12, 18))
axes = axes.flatten()

for i, column in enumerate(columns_to_plot):
    sns.histplot(bike_df[column], color='#23B6EA', ax=axes[i])
    axes[i].set_yscale('log')
    axes[i].set_title(f'Distribution of "{column}"')
    axes[i].set_xlabel(column)
    axes[i].set_ylabel('Frequency')
    if bike_df[column].dtype != object:
        mean_value = bike_df[column].mean()
        median_value = bike_df[column].median()
        std_value = bike_df[column].std()
        summary_text = f"Mean: {mean_value:.2f}\nMedian: {median_value:.2f}\nStd: {std_value:.2f}"
        axes[i].text(0.75, 0.8, summary_text, transform=axes[i].transAxes, fontsize=10, verticalalignment='baseline',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
        #text.get_bbox_patch().set_alpha(0.5)
        
plt.tight_layout()
#plt.show()
#fig.savefig("1D_vis.png", bbox_inches='tight')

In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(bike_df['registered'], color='firebrick', binwidth=20, label='Registered')
sns.histplot(bike_df['casual'], color='#23B6EA', label='Casual', binwidth=20)
plt.xlabel('Number of Bike Rentals')
plt.ylabel('Frequency')
plt.legend()
plt.title('Distribution of Casual vs. Registered Bike Rentals')
plt.savefig("casual_registered_frequency.png", bbox_inches='tight')
plt.show()

# Bivariate visualizations 

In [ ]:
bike_df = pd.read_csv(path_to_bikes)

columns = [i for i in bike_df.columns[2:]]
#columns.remove('casual')
#columns.remove('registered')
correlation_matrix = bike_df[columns].corr() 
fig, ax = fig, ax = plt.subplots(figsize=(18, 16))

sns.heatmap(correlation_matrix, annot=True, cmap='icefire', vmin=-1, vmax=1, linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()
fig.savefig("Correlation_Matrix.png", bbox_inches='tight')

In [ ]:
bike_df = pd.read_csv(path_to_bikes)
mapping = {
    'holiday': {0: "no", 1: "yes"},
    'workingday': {0: "no", 1: "yes"},
    'weathersit' : {1 : "Clear", 2 : "Mist", 3 : "Light precip.", 4 : "High precip."}, #precipitation
    'season': {1: "Winter", 2: "Spring", 3: "Summer", 4: "Autumn"},
    'weekday': {0: "Sunday", 1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday"}
}
for i in mapping.keys():
    bike_df[i] = bike_df[i].map(mapping[i])
    
selected_weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]  

g = sns.FacetGrid(bike_df[bike_df['weekday'].isin(selected_weekdays)], col='weekday', row='holiday',
                  height=4, aspect=1.2)

g.map_dataframe(sns.histplot, x='hr', y='cnt', binwidth=(1,20), color ='#000022', bins=4, cbar=True)

plt.show()

In [ ]:
bike_df = pd.read_csv(path_to_bikes)
mapping = {
    'holiday': {0: "no", 1: "yes"},
    'workingday': {0: "no", 1: "yes"},
    'weathersit' : {1 : "Clear", 2 : "Mist", 3 : "Light precip.", 4 : "High precip."}, #precipitation
    'season': {1: "Winter", 2: "Spring", 3: "Summer", 4: "Autumn"},
    'weekday': {0: "Sunday", 1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday"}
}
for i in mapping.keys():
    bike_df[i] = bike_df[i].map(mapping[i])
    
selected_weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"] 
g = sns.FacetGrid(bike_df[bike_df['weekday'].isin(selected_weekdays)], col='weekday', row='holiday',
                  height=4, aspect=1.2)
f = sns.FacetGrid(bike_df[bike_df['weekday'].isin(selected_weekdays)], col='weekday', row='holiday',
                  height=4, aspect=1.2)
g.map_dataframe(sns.histplot, x='hr', y='registered', color='firebrick', binwidth=(1,20), bins=4, cbar=True)
f.map_dataframe(sns.histplot, x='hr', y='casual', color='#23B6EA', binwidth=(1,20), bins=4, cbar=True)
#g.savefig("Worksday_r.png", bbox_inches='tight')
#f.savefig("Worksday_c.png", bbox_inches='tight')
plt.show()

In [ ]:
selected_weekdays = ["Saturday", "Sunday"] 

g = sns.FacetGrid(bike_df[bike_df['weekday'].isin(selected_weekdays)], col='weekday',
                  height=4, aspect=1.2)
f = sns.FacetGrid(bike_df[bike_df['weekday'].isin(selected_weekdays)], col='weekday',
                  height=4, aspect=1.2)

g.map_dataframe(sns.histplot, x='hr', y='registered', color='firebrick', binwidth=(1,20), bins=4, cbar=True)
f.map_dataframe(sns.histplot, x='hr', y='casual', color='#23B6EA', binwidth=(1,20), bins=4, cbar=True)

g.savefig("Weekend_r.png", bbox_inches='tight')
f.savefig("Weekend_c.png", bbox_inches='tight')

plt.show()

In [ ]:
selected_weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"] 
g = sns.FacetGrid(bike_df[bike_df['weekday'].isin(selected_weekdays)], col='weathersit', row='season',
                  height=4, aspect=1.2)
f = sns.FacetGrid(bike_df[bike_df['weekday'].isin(selected_weekdays)], col='weathersit', row='season',
                  height=4, aspect=1.2)
g.map_dataframe(sns.histplot, x='hr', y='registered', color='firebrick', binwidth=(1,20), bins=4, cbar=True)
f.map_dataframe(sns.histplot, x='hr', y='casual', color='#23B6EA', binwidth=(1,20), bins=4, cbar=True)
#g.savefig("Weathersit_r.png", bbox_inches='tight')
#f.savefig("Weathersit_c.png", bbox_inches='tight')
plt.show()

In [ ]:
g = sns.FacetGrid(bike_df, row='workingday', col='season', height=4, aspect=1.2)
f = sns.FacetGrid(bike_df, row='workingday', col='season', height=4, aspect=1.2)
g.map_dataframe(sns.histplot, x='temp', y='registered', color='firebrick', binwidth=(0.05,20), bins=4, cbar=True)
f.map_dataframe(sns.histplot, x='temp', y='casual', color='#23B6EA', binwidth=(0.05,20), bins=4, cbar=True)
#g.savefig("season_workingday_r.png", bbox_inches='tight')
#f.savefig("season_workingday_c.png", bbox_inches='tight')
#g.set_axis_labels("Hour (hr)", "Usage")
plt.show()

In [ ]:
bike_df = pd.read_csv(path_to_bikes)

bike_df['date'] = pd.to_datetime(bike_df['dteday'])
bike_df.set_index('date', inplace=True)
weekly_sum = bike_df[['casual', 'registered']].resample('W').sum()
weekly_sum['week'] = range(1, len(weekly_sum) + 1)
weekly_sum['casual_to_registered_ratio'] = weekly_sum['casual'] / weekly_sum['registered']

plt.figure(figsize=(15, 6))

slope, intercept, r_value, p_value, std_err = stats.linregress(weekly_sum['week'], weekly_sum['casual_to_registered_ratio'])
legend_text = f'y = {slope:.4f}x + {intercept:.4f}\nR-squared = {r_value**2:.4f}'
sns.regplot(x='week', y='casual_to_registered_ratio', data=weekly_sum, ci=None, scatter=False, color='red')

plt.legend([legend_text], loc='upper right', fontsize=12, facecolor='lightgray')

sns.lineplot(x='week', y='casual_to_registered_ratio', data=weekly_sum, errorbar=None, color='#23B6EA')

plt.xlabel('Week number')
plt.ylabel('Weekly Casual-to-Registered Ratio')
plt.title('Casual-to-Registered Ratio Over Weeks')
plt.savefig("casual_registered_ration.png", bbox_inches='tight')
plt.grid(True)
plt.show()

# Data clustering

In [ ]:
pip install prince

In [ ]:
import prince
bike_df = pd.read_csv(path_to_bikes)

### FAMD

In [ ]:
FAMD_data = bike_df[[i for i in bike_df.columns[2:-3]]]
famd = prince.FAMD(n_components = 5, n_iter = 5, random_state = 505);
famd = famd.fit(FAMD_data);
famd.column_coordinates_

In [ ]:
famd.row_coordinates(FAMD_data).head()

### Clustering

In [ ]:
clustering_data = famd.row_coordinates(FAMD_data).copy()
inertias = []
for k in range(1, 25):
    kmeans = KMeans(n_clusters=k, random_state=505, n_init="auto")
    kmeans.fit(clustering_data)
    inertias.append(kmeans.inertia_)
plt.plot(range(1, 25), inertias, marker='o',) 
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()

In [ ]:
n_clusters = 9
clustering_data = famd.row_coordinates(FAMD_data).copy()
c_d_w_c = famd.row_coordinates(FAMD_data).copy()

kmeans = KMeans(n_clusters=n_clusters, random_state=505, n_init="auto")
kmeans.fit(clustering_data)

c_d_w_c['cluster'] = kmeans.labels_

In [ ]:
sns.set(style="ticks")
fig = sns.pairplot(c_d_w_c,  hue="cluster", palette="rainbow")
plt.show()
#fig.savefig("famd_cluster.png", bbox_inches='tight')
#fig.savefig("famd_cluster.pdf", bbox_inches='tight')

In [ ]:
vis_data = bike_df[[i for i in bike_df.columns[2:]]].copy()
vis_data['cluster'] = c_d_w_c['cluster']

In [ ]:
sns.set(style="ticks")
fig = sns.pairplot(vis_data,  hue="cluster", palette="rainbow")
plt.show()
#fig.savefig("vis_cluster.png", bbox_inches='tight')
#fig.savefig("vis_cluster.pdf", bbox_inches='tight')

In [ ]:
for cluster_num in range(n_clusters):
    fig, axes = plt.subplots(2, 5, figsize=(18, 6))
    for i in range(len(bike_df.columns[4:-3])):

        cluster_data = vis_data[vis_data['cluster'] == cluster_num]
        bike_df.columns[2:-3][i]
        row = i // 5
        col = i % 5   
        axes[row, col].scatter(cluster_data[bike_df.columns[4:-3][i]], cluster_data['registered'], c='firebrick', s=1, alpha=0.5)
        axes[row, col].scatter(cluster_data[bike_df.columns[4:-3][i]], cluster_data['casual'], c='#23B6EA', s=1, alpha=0.5)
        axes[row, col].set_title(f'Cluster {cluster_num}')
        axes[row, col].set_xlabel(bike_df.columns[4:-3][i])
        axes[row, col].set_ylabel('Number of Rental Bikes')

    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()
    #fig.savefig(f'reg_cas_{cluster_num}.png', bbox_inches='tight')

### Reverse clustering

In [ ]:
data = bike_df[bike_df.columns[-3:-1]].copy()
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
scaled_df = pd.DataFrame(scaled_data, columns=data.columns)
print(scaled_df.head())

In [ ]:
clustering_data_rev = scaled_df
inertias = []
for k in range(1, 25):
    kmeans = KMeans(n_clusters=k, random_state=505, n_init="auto")
    kmeans.fit(clustering_data_rev)
    inertias.append(kmeans.inertia_)
plt.plot(range(1, 25), inertias, marker='o',) 
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method')
plt.show()

In [ ]:
n_clusters_ = 5
clustering_data_rev = scaled_df.copy()
c_d_w_c_rev = scaled_df.copy()

kmeans = KMeans(n_clusters=n_clusters_, random_state=505, n_init="auto")
kmeans.fit(clustering_data_rev)

c_d_w_c_rev['cluster'] = kmeans.labels_
data['cluster'] = kmeans.labels_

In [ ]:
sns.set(style="ticks")
fig = sns.pairplot(data,  hue="cluster", palette="rainbow")
plt.show()
#fig.savefig("famd_cluster.png", bbox_inches='tight')
#fig.savefig("famd_cluster.pdf", bbox_inches='tight')

In [ ]:
data_full = bike_df[bike_df.columns[2:]].copy()
data_full['cluster'] = kmeans.labels_
sns.set(style="ticks")
fig = sns.pairplot(data_full,  hue="cluster", palette="rainbow")
plt.show()
#fig.savefig("rev_cluster.png", bbox_inches='tight')
#fig.savefig("rev_cluster.pdf", bbox_inches='tight')

In [ ]:
features = ['hr', 'workingday', 'atemp', 'casual', 'registered', 'cluster']
fig = sns.pairplot(data_full[data_full["cluster"]==0][features],  hue="cluster", palette="rainbow")
plt.show()

# Forecast model

In [ ]:
pip install livelossplot

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import seaborn as sns 
import livelossplot as llp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from keras.constraints import NonNeg
import math
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
bikes_df = pd.read_csv(path_to_bikes)
feature_columns = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
targets = ['casual', 'registered', 'cnt']

X = bikes_df[feature_columns]
y = bikes_df[targets]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=505)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
def custom_relu(x):
    return tf.keras.activations.relu(x, max_value=None)

def create_model(input_shape, hidden_units=(128, 64, 32), activation="relu", regularizer=l2(0.01), dropout_rate=0.0):

    input_layer = keras.layers.Input(shape=input_shape)
    hidden_layer = input_layer

    for units in hidden_units:
        hidden_layer = keras.layers.Dense(units, activation=activation, kernel_regularizer=regularizer)(hidden_layer)
        hidden_layer = keras.layers.BatchNormalization()(hidden_layer)
        if dropout_rate > 0.0:
            hidden_layer = tf.keras.layers.Dropout(dropout_rate)(hidden_layer)

    output_casual = Dense(1, name='casual', activation=custom_relu, kernel_constraint=NonNeg())(hidden_layer)
    output_registered = Dense(1, name='registered', activation=custom_relu, kernel_constraint=NonNeg())(hidden_layer)
 
    model = keras.Model(inputs=input_layer, outputs=[output_casual, output_registered])

    return model

In [ ]:
model = create_model(input_shape=(X_train_scaled.shape[1],), hidden_units=(128, 64, 32), dropout_rate=0.0)
model.summary()

In [ ]:
METRICS = [
    tf.keras.metrics.MeanSquaredError(name='mse'),
    tf.keras.metrics.MeanSquaredError(name='mse'),
    #tf.keras.metrics.RootMeanSquaredError(name='rmse'),
    #tf.keras.metrics.MeanAbsoluteError(name='mae'),
]

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-8,
    decay_steps=1000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch / 20))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=METRICS)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled,
    [y_train['casual'].to_numpy(), y_train['registered'].to_numpy()],
    epochs=1000,
    batch_size=32,
    validation_split=0.2,
    verbose=2,
    callbacks=[early_stopping, lr_scheduler]
)

In [ ]:
y_pred_casual, y_pred_registered = model.predict(X_test_scaled)

mae_cnt = tf.keras.metrics.mean_absolute_error(y_test['cnt'], y_pred_casual.flatten()+y_pred_registered.flatten()).numpy()
mae_casual = tf.keras.metrics.mean_absolute_error(y_test['casual'], y_pred_casual.flatten()).numpy()
mae_registered = tf.keras.metrics.mean_absolute_error(y_test['registered'], y_pred_registered.flatten()).numpy()

print(f"Mean Absolute Error for 'cnt': {mae_cnt}")
print(f"Mean Absolute Error for 'casual': {mae_casual}")
print(f"Mean Absolute Error for 'registered': {mae_registered}")

plt.semilogx(history.history["learning_rate"], history.history["loss"])

In [ ]:
model = create_model(input_shape=(X_train_scaled.shape[1],), hidden_units=(128, 64, 32), dropout_rate=0.0)

METRICS={
    'casual': [
        tf.keras.metrics.MeanSquaredError(name='mse'),
        #tf.keras.metrics.RootMeanSquaredError(name='rmse'),
        tf.keras.metrics.MeanAbsoluteError(name='mae')
    ],
    'registered': [
        tf.keras.metrics.MeanSquaredError(name='mse'),
        #tf.keras.metrics.RootMeanSquaredError(name='rmse'),
        tf.keras.metrics.MeanAbsoluteError(name='mae')
    ]
}

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),#'adam', 
              loss='mean_squared_error', 
              metrics=METRICS)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=20,
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled,
    [y_train['casual'].to_numpy(), y_train['registered'].to_numpy()],
    epochs=500,
    batch_size=32,
    validation_split=0.2,
    verbose=2,
    callbacks=[early_stopping, llp.PlotLossesKerasTF(outputs=[llp.outputs.MatplotlibPlot(cell_size=(4, 3))])]
)

y_pred_casual, y_pred_registered = model.predict(X_test_scaled)

mae_casual = tf.keras.metrics.mean_absolute_error(y_test['casual'], y_pred_casual.flatten()).numpy()
mae_registered = tf.keras.metrics.mean_absolute_error(y_test['registered'], y_pred_registered.flatten()).numpy()
mae_cnt = tf.keras.metrics.mean_absolute_error(y_test['cnt'], y_pred_casual.flatten()+y_pred_registered.flatten()).numpy()

print(f"Mean Absolute Error for 'cnt': {mae_cnt}")
print(f"Mean Absolute Error for 'casual': {mae_casual}")
print(f"Mean Absolute Error for 'registered': {mae_registered}")

In [ ]:
Vis_Test_a = pd.concat([X_test, y_test], axis=1)
Vis_Test_a['type'] = ['true']*len(Vis_Test_a)
a = [i for i in range(1, len(Vis_Test_a) + 1)]
Vis_Test_a = Vis_Test_a.set_index(pd.Index(a))

Vis_Test_b = X_test
Vis_Test_b['casual'] = y_pred_casual.flatten()
Vis_Test_b['registered'] = y_pred_registered.flatten()
Vis_Test_b['cnt'] = y_pred_casual.flatten() + y_pred_registered.flatten()
Vis_Test_b['type'] = ['prediction']*len(Vis_Test_b)
b = [i for i in range(len(Vis_Test_b) + 1, 2*len(Vis_Test_b) + 1)]
Vis_Test_b = Vis_Test_b.set_index(pd.Index(b))

Vis_Test = pd.concat([Vis_Test_a, Vis_Test_b])

In [ ]:
g = sns.FacetGrid(Vis_Test, col="type", row='workingday', height=4, aspect=1.2)
g.map_dataframe(sns.histplot, x='hr', y='cnt', binwidth=(1,20), color ='#000022', bins=4, cbar=True)
g.savefig("workingday_true_pred_cnt.png", bbox_inches='tight')
plt.show()

In [ ]:
g = sns.FacetGrid(Vis_Test, col="type", row='workingday', height=4, aspect=1.2)
g.map_dataframe(sns.histplot, x='hr', y='casual', binwidth=(1,20), color ='#23B6EA', bins=4, cbar=True)
g.savefig("workingday_true_pred_casual.png", bbox_inches='tight')
plt.show()

In [ ]:
g = sns.FacetGrid(Vis_Test, col="type", row='workingday', height=4, aspect=1.2)
g.map_dataframe(sns.histplot, x='hr', y='cnt', binwidth=(1,20), color = 'firebrick', bins=4, cbar=True)
g.savefig("workingday_true_pred_registered.png", bbox_inches='tight')
plt.show()


In [ ]:
X = bikes_df[feature_columns]
y = bikes_df[['casual', 'registered']] 

kf = KFold(n_splits=5, shuffle=True, random_state=42)

mae_scores = []

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=20,
    restore_best_weights=True)
    
    model = create_model(input_shape=(X_train_scaled.shape[1],), hidden_units=(128, 64, 32), dropout_rate=0.0)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),#'adam', 
              loss='mean_squared_error', 
              metrics=METRICS)
    
    model.fit(
        X_train_scaled,
        [y_train['casual'].to_numpy(), y_train['registered'].to_numpy()],
        epochs=500,
        batch_size=32,
        validation_split=0.2,
        verbose=2,
        callbacks=[early_stopping],)

    y_pred_casual, y_pred_registered = model.predict(X_test_scaled)
    fold_mae_casual = mean_absolute_error(y_test['casual'].to_numpy(), y_pred_casual)
    fold_mae_registered = mean_absolute_error(y_test['registered'].to_numpy(), y_pred_registered)
    mae_scores.append([fold_mae_casual, fold_mae_registered])
    print(fold_mae_casual, fold_mae_registered)

In [ ]:
mae_scores = np.array(mae_scores)
baseline_mae_casual = mean_absolute_error(y['casual'], np.full_like(y['casual'], y['casual'].mean()))
baseline_mae_registered = mean_absolute_error(y['registered'], np.full_like(y['registered'], y['registered'].mean()))

print(f"Cross-Validated MAE: casual {mae_scores[:, 0]}, registered {mae_scores[:, 1]}")
print(f"Mean MAE: casual {np.mean(mae_scores[:, 0])}, registered {np.mean(mae_scores[:, 1])}")
print(f"Baseline MAE: casual {baseline_mae_casual}, registered {baseline_mae_registered}")

if (np.mean(mae_scores[:, 0]) < baseline_mae_casual) & (np.mean(mae_scores[:, 1]) < baseline_mae_registered):
    print("The model performs better than the baseline.")
else:
    print("The model does not perform better than the baseline.")